## <p align=center>Web Scraping de los viajes en taxi en Manhattan, NY</p>
### <p align=center>Período: 2016-2019</p>

In [15]:
# Importación de librerías

import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from google.cloud import storage
import os

In [6]:
# URL del sitio web

url = 'https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page'

In [7]:
# Instancia de la clase requests

response = requests.get(url)

# Creación de un objeto BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')

# Lista vacía para almacenar los enlaces

links = []

# Búsqueda de la clase 'faq-answers' en el objeto soup

faq_answers = soup.find_all('div', class_='faq-answers')

# Bucle for para obtener los enlaces con parámetros específicos (extensión .parquet, años 2016 a 2019, yellow taxis)

for div in faq_answers:
    for a in div.find_all('a', href=True):
        href = a['href']
        if href.endswith('.parquet') and 'yellow' in href and re.search(r'201[6-9]', href):
            
            # Agregar los enlaces a la lista

            links.append(href)

df = pd.DataFrame(links, columns=['links'])

In [10]:
df.head(1)

,links
0,https://d37ci6vzurychx.cloudfront.net/trip-dat...


In [11]:
df.shape

(48, 1)

In [16]:
# Configuración de la autenticación de GCP

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'datalogic-taxis-ny-4d9c91e757e1.json'

# Creación de un cliente de GCP

client = storage.Client()
bucket_name = 'prueba_datalogic'
bucket = client.bucket(bucket_name)

for url in df['links']:
    file_name = url.split('/')[-1]
    response = requests.get(url)
    
    if response.status_code == 200:

        # Guardamos el archivo localmente
        with open(file_name, 'wb') as f:
            f.write(response.content)

        # Leemos el archivo con pandas y seleccionamos las columnas que nos interesan
        df = pd.read_parquet(file_name)
        df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type', 'total_amount']]

        # Renombramos las columnas traduciendo los nombres
        df.rename(columns={'tpep_pickup_datetime': 'fecha_hora_subida', 'tpep_dropoff_datetime': 'fecha_hora_bajada', 'passenger_count': 'cantidad_pasajeros', 'trip_distance': 'distancia_recorrida', 'PULocationID': 'zona_subida', 'DOLocationID': 'zona_bajada', 'payment_type': 'tipo_pago', 'total_amount': 'monto_total'}, inplace=True)

        # Guardamos el archivo con los datos limpios
        df.to_parquet(file_name, index=False)
        
        # Lo subimos a Cloud Storage
        blob = bucket.blob(file_name)
        blob.upload_from_filename(file_name)

        # Eliminamos el archivo local
        os.remove(file_name)

        print(f'Archivo {file_name} subido a Cloud Storage')

        # Como estamos haciendo una prueba, detenemos el bucle después de subir un archivo
        break

    else:
        print(f'Error en la descarga del archivo {file_name}')

Archivo yellow_tripdata_2019-01.parquet subido a Cloud Storage


In [19]:
for url in df['links']:
    file_name = url.split('/')[-1]
    response = requests.get(url)
    
    if response.status_code == 200:

        # Guardamos el archivo localmente
        with open(file_name, 'wb') as f:
            f.write(response.content)

        # Leemos el archivo con pandas y seleccionamos las columnas que nos interesan
        df = pd.read_parquet(file_name)
        df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type', 'total_amount']]

        # Renombramos las columnas traduciendo los nombres
        df.rename(columns={'tpep_pickup_datetime': 'fecha_hora_subida', 'tpep_dropoff_datetime': 'fecha_hora_bajada', 'passenger_count': 'cantidad_pasajeros', 'trip_distance': 'distancia_recorrida', 'PULocationID': 'zona_subida', 'DOLocationID': 'zona_bajada', 'payment_type': 'tipo_pago', 'total_amount': 'monto_total'}, inplace=True)

        # Guardamos el archivo con los datos limpios
        df.to_parquet(file_name, index=False)

        break

In [20]:
df.head(1)

,fecha_hora_subida,fecha_hora_bajada,cantidad_pasajeros,distancia_recorrida,zona_subida,zona_bajada,tipo_pago,monto_total
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,151,239,1,9.95
